# Object model

CoHDLs object model is a compromise between VHDL and Python. It splits all objects into two groups:

* type qualified objects
    * runtime variable in synthesizable contexts
    * equivalent to VHDLs Signal/Variable declarations
    * wrap synthesizable types
    * are the only objects, that can be assigned new values
    * are the only objects, that end up in the VHDL representation
* all other objects
    * constant in synthesizable contexts (except in `__init__` methods to allow the construction of local objects)
    * are used to structure type qualified objects and metadata

## references

In Python assignments using the `=` operator do not copy values. Instead the name at the left hand side becomes a reference to the object at the right hand side. The same is true in synthesizable contexts, with the additional limitation that defined references can not be overwritten. Every variable name can only be defined once per function (for loops and generator expressions are exceptions to this rule).

All references to the same object are aliases and interchangeable without affecting the resulting VHDL. As in normal Python function arguments, list/dict elements and class members are also implemented as references.

## assignments

Only type qualified objects can be assigned new values at runtime. CoHDL defines three different operators for this purpose

* signal assignment `<<=`
* push assignment `^=`
* variable assignment `@=`

There are no other overloaded assignment operators (`+=`, `>>=`, `|=` and so on are not allowed).

---
# Type qualifiers

Type qualifiers are applied to synthesizable types (see next section) to produce Signal/Variable/Temporary types.

In [1]:
from cohdl import Signal, Variable, Bit, Temporary

# 'a' is an integer value
a = 1

# 'b' is a Bit object
# (cohdl treats this as a constant/literal just like 'a')
b = Bit(1)

# 'c' Signal of type Bit with no default value
c = Signal[Bit]()
# 'd' Variable of type bool with default value True
d = Variable[bool](True)
# 'e' Temporary of type int with value 123
e = Temporary[int](123)

There are four different type qualifiers. **Signals**, **Ports** and **Variables** are translated into their VHDL equivalents. **Temporaries** are returned by expressions operating on type qualified objects and turned into VHDL Signals or Variables depending on the context (because Variables are not allowed in concurrent contexts and Signal assignment is delayed in sequential contexts).

* Signal/Port
    
    Signals can be shared by multiple synthesizable contexts but may only be driven by one. Signals support two types of assignment:

    * signal assignment (`<<=`, `.next`)

        Is equivalent to VHDLs `<=` operator. When used in sequential contexts, the value is updated once the end of the context is reached.
    * push assignment (`^=`, `.push`)

        Sets the value of a signal for one clock cycle. When the CoHDL compiler encounters a push assignment it inserts a reset expression (assignment of the default value) at the start of the context. This operator is limited to sequential contexts and only defined for signals with default values. Push- and next-assignment are mutually exclusive, they can not be applied to the same object.

    Ports are used to define inputs and outputs of entities and may only be declared at entity scope. Apart from an additional direction property, they are normal Signals. It is not possible to write to input ports. Unlike VHDL, CoHDL allows to read from outputs, the compiler inserts buffer signals (it does not change the port type to buffer).

* Variable (`@=`, `.value`)

    Variables can only be used in sequential contexts. They use the `@=` operator or the equivalent `.value` property for assignment and update their value immediately.
* Temporary

    All operations that depend on type qualified objects return Temporaries. It is also possible to construct them explicitly like Signals and Variables. Temporaries are constant for their entire lifetime, it is not possible to assign new values to them.
    
    Inside coroutines CoHDL restricts the usage of Temporaries to a single state. The idea behind this is to limit the lifetime of temporary objects and prevent registers from being inferred. Ideally Temporaries should never turn into latches/flip-flops, but this is currently not enforced in every circumstance.

In [2]:
from cohdl import Entity, Port, Signal, Variable, Bit
from cohdl import std

class AssignmentExample(Entity):
    clk = Port.input(Bit)

    data = Port.input(Bit)
    update = Port.input(Bit)

    def architecture(self):
        v = Variable[Bit]()
        s = Signal[Bit]()

        # s_push has a default value of '0' (== False)
        s_push = Signal[Bit](False)

        @std.sequential(std.Clock(self.clk))
        def logic_operators():
            # this is not required when the properties are used
            # instead of the assignment operators
            nonlocal s, s_push, v

            if self.update:
                # normal signal assignment
                # assigning to the .next property is equivalent to the '<<=' operator
                s <<= self.data
                s.next = self.data

                # cohdl assigns the default value to s_push
                # at the start of the sequential context to implement
                # this operator
                s_push ^= self.data
                s_push.push = self.data

                # variable assignment used the @= operator
                # or the .value property
                v @= self.data
                v.value = self.data
        

print(std.VhdlCompiler.to_string(AssignmentExample))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity AssignmentExample is
  port (
    clk : in std_logic;
    data : in std_logic;
    update : in std_logic
    );
end AssignmentExample;


architecture arch_AssignmentExample of AssignmentExample is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal s_push : std_logic := '0';
  signal s : std_logic;
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  

  logic_operators: process(clk)
    variable temp : boolean;
    variable v : std_logic;
  begin
    if rising_edge(clk) then
      s_push <= '0';
      temp := update = '1';
      if temp then
        s <= data;
        s <= data;
        s_push <= data;
        s_push <= data;
        v := data;
        v := data;
      end if;
    end if;
  end process;
end architecture arch_AssignmentExample;
